In [2]:
import joblib
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

In [ ]:
df = joblib.load(r"..\assets\RAWG\games_data_with_id.pkl")
df = df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62138 entries, 0 to 704056
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  62138 non-null  Int64  
 1   slug                62138 non-null  string 
 2   name                62138 non-null  string 
 3   released            58139 non-null  string 
 4   tba                 62138 non-null  boolean
 5   background_image    62102 non-null  string 
 6   rating              62138 non-null  Float64
 7   rating_top          62138 non-null  Int64  
 8   ratings             62138 non-null  object 
 9   ratings_count       62138 non-null  Int64  
 10  reviews_text_count  62138 non-null  Int64  
 11  added               62138 non-null  Int64  
 12  added_by_status     32004 non-null  object 
 13  metacritic          4247 non-null   Int64  
 14  playtime            62138 non-null  Int64  
 15  suggestions_count   62138 non-null  Int64  
 16  updated 

In [4]:
df.sample(5)

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,user_game,reviews_count,saturated_color,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating,steam_app_id
387462,675346,infinite-progression,Infinite Progression,2021-10-05,False,https://media.rawg.io/media/screenshots/4e6/4e...,0.0,0,[],0,0,0,None,<NA>,0,5,2021-11-22T03:30:26,<NA>,0,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 40, 'name': 'Casual', 'slug': 'casual'...","[{'id': 671246, 'store': {'id': 9, 'name': 'it...",<NA>,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0,1794710
17435,15961,buildmorecubes,BuildMoreCubes,2016-11-21,False,https://media.rawg.io/media/screenshots/c8c/c8...,0.0,0,[],0,0,77,"{'yet': 1, 'owned': 75, 'toplay': 1}",<NA>,4,289,2019-08-28T22:11:16,<NA>,0,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 17600, 'store': {'id': 1, 'name': 'Ste...",<NA>,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0,557190
224592,861333,streamer-future-wars,Streamer Future Wars,2022-10-03,False,https://media.rawg.io/media/screenshots/bcf/bc...,0.0,0,[],0,0,0,None,<NA>,0,138,2022-10-07T04:31:04,<NA>,0,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 5, 'name': 'RPG', 'slug': 'role-playin...","[{'id': 858749, 'store': {'id': 1, 'name': 'St...",<NA>,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0,2122130
115689,987260,zhong-fu-yong-zhe-chuan,種付勇者傳,2024-08-13,False,https://media.rawg.io/media/screenshots/3cc/3c...,0.0,0,[],0,0,0,None,<NA>,0,319,2024-08-16T13:39:37,<NA>,0,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 3, 'name': 'Adventure', 'slug': 'adven...","[{'id': 988211, 'store': {'id': 1, 'name': 'St...",<NA>,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0,3007270
2271,16869,faerie-solitaire,Faerie Solitaire,2010-09-17,False,https://media.rawg.io/media/screenshots/952/95...,3.01,3,"[{'id': 3, 'title': 'meh', 'count': 36, 'perce...",90,0,1234,"{'yet': 19, 'owned': 1143, 'beaten': 40, 'topl...",<NA>,5,232,2025-01-14T01:08:47,<NA>,90,0f0f0f,0f0f0f,"[{'platform': {'id': 5, 'name': 'macOS', 'slug...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 40, 'name': 'Casual', 'slug': 'casual'...","[{'id': 18711, 'store': {'id': 1, 'name': 'Ste...",<NA>,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",<NA>,38600


In [5]:
# game released date
df['released'] = pd.to_datetime(df['released'])
df['released_year'] = df['released'].dt.year
df['released_month'] = df['released'].dt.month
df['released_day'] = df['released'].dt.day

In [6]:
# RAWG user rating
def extract_rating_percentages(ratings_list):
    ratings_dict = {5: 0.0, 4: 0.0, 3: 0.0, 1: 0.0}  # Default values

    for rating in ratings_list:
        if rating['id'] in ratings_dict:
            ratings_dict[rating['id']] = float(rating['percent'])

    return pd.Series([
        ratings_dict[5], ratings_dict[4], ratings_dict[3], ratings_dict[1]
    ])

df[['RAWG_rating_5_percent', 'RAWG_rating_4_percent', 'RAWG_rating_3_percent', 'RAWG_rating_1_percent']] = df['ratings'].apply(extract_rating_percentages)
df.rename(columns={'rating': 'weighted_avg_RAWG_rating'}, inplace=True)

In [7]:
# bookmark by RAWG user
df.rename(columns={'added': 'RAWG_bookmark_count'}, inplace=True)

def extract_added_status(added_info):
    if not isinstance(added_info, dict):  # Handle missing or invalid data
        return pd.Series({}, dtype="float64")

    extracted_data = {f"bookmark_type_{key}_count": int(value) for key, value in added_info.items()}

    return pd.Series(extracted_data, dtype="float64")

added_status_df = df["added_by_status"].apply(extract_added_status)
df = pd.concat([df, added_status_df], axis=1)

In [8]:
df.rename(columns={'metacritic': 'metacritic_rating',
                   'playtime': 'steam_avg_playtime(h)',
                   'suggestions_count': 'RAWG_system_suggest_count'
                   }, inplace=True)

In [9]:
# platform
def extract_platform_list(platforms_list):
    if not isinstance(platforms_list, list):  # Handle missing or invalid data
        return []
    
    return [platform_entry.get("platform", {}).get("name", "") for platform_entry in platforms_list if "platform" in platform_entry]

df["available_platform"] = df["platforms"].apply(extract_platform_list)
df["available_parent_platforms"] = df["parent_platforms"].apply(extract_platform_list)

In [10]:
# genre
def extract_genre_list(genre_list):
    if not isinstance(genre_list, list):  # Handle missing or invalid data
        return []
    
    return [genre_entry.get("name") for genre_entry in genre_list]

df["genres"] = df["genres"].apply(extract_genre_list)

In [11]:
# tag
def extract_tags_list(tag_list):
    if not isinstance(tag_list, list):  # Handle missing or invalid data
        return []
    
    return [tag_entry.get("name") for tag_entry in tag_list]

df["tags"] = df["tags"].apply(extract_tags_list)

In [12]:
df["esrb_rating"] = df["esrb_rating"].apply(lambda x: x.get('name', 'Missing') if not x is None else 'Missing')

In [13]:
df.drop(columns=['released', 'background_image', 'ratings', 'rating_top', 'updated', 'user_game', 'saturated_color', 'added_by_status',
                 'dominant_color', 'platforms', 'parent_platforms', 'stores', 'clip', 'short_screenshots', 'community_rating'], inplace=True)

In [14]:
df.sample(5)

,id,slug,name,tba,weighted_avg_RAWG_rating,ratings_count,reviews_text_count,RAWG_bookmark_count,metacritic_rating,steam_avg_playtime(h),RAWG_system_suggest_count,reviews_count,genres,tags,esrb_rating,steam_app_id,released_year,released_month,released_day,RAWG_rating_5_percent,RAWG_rating_4_percent,RAWG_rating_3_percent,RAWG_rating_1_percent,bookmark_type_yet_count,bookmark_type_owned_count,bookmark_type_beaten_count,bookmark_type_toplay_count,bookmark_type_dropped_count,bookmark_type_playing_count,available_platform,available_parent_platforms
61800,984489,xiu-xian-shen-jue,修仙神诀,False,0.0,0,0,3,<NA>,0,74,0,"[RPG, Strategy, Casual]","[Multiplayer, RPG, Co-op, cooperative, Online ...",Missing,3029670,2024.0,6.0,27.0,0.00,0.0,0.00,0.00,NaN,3.0,NaN,NaN,NaN,NaN,[PC],[PC]
350339,718083,small-war-2,Small War,False,0.0,0,0,0,<NA>,0,264,0,"[RPG, Strategy, Simulation, Casual, Indie]","[Singleplayer, Steam Achievements, Multiplayer...",Missing,1208410,NaN,NaN,NaN,0.00,0.0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,[PC],[PC]
14637,21526,legends-of-atlantis-exodus,Legends of Atlantis: Exodus,False,0.0,1,0,109,<NA>,6,458,1,"[Strategy, Casual]","[Singleplayer, Steam Achievements, Steam Cloud...",Missing,347520,2015.0,9.0,3.0,0.00,0.0,100.00,0.00,NaN,109.0,NaN,NaN,NaN,NaN,[PC],[PC]
6908,14510,crab-dub,Crab Dub,False,2.33,6,0,314,<NA>,3,134,6,"[Casual, Indie]","[Singleplayer, Steam Achievements, steam-tradi...",Missing,539450,2017.0,1.0,19.0,16.67,0.0,33.33,50.00,8.0,302.0,1.0,1.0,2.0,NaN,[PC],[PC]
6267,16307,abc-coloring-town,ABC Coloring Town,False,1.36,11,0,359,<NA>,4,259,11,"[Casual, Educational]","[Singleplayer, steam-trading-cards, Family Fri...",Everyone 10+,478010,2013.0,2.0,23.0,0.00,0.0,18.18,81.82,4.0,350.0,1.0,NaN,4.0,NaN,"[PC, iOS]","[PC, iOS]"


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62138 entries, 0 to 704056
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           62138 non-null  Int64  
 1   slug                         62138 non-null  string 
 2   name                         62138 non-null  string 
 3   tba                          62138 non-null  boolean
 4   weighted_avg_RAWG_rating     62138 non-null  Float64
 5   ratings_count                62138 non-null  Int64  
 6   reviews_text_count           62138 non-null  Int64  
 7   RAWG_bookmark_count          62138 non-null  Int64  
 8   metacritic_rating            4247 non-null   Int64  
 9   steam_avg_playtime(h)        62138 non-null  Int64  
 10  RAWG_system_suggest_count    62138 non-null  Int64  
 11  reviews_count                62138 non-null  Int64  
 12  genres                       62138 non-null  object 
 13  tags                

In [ ]:
joblib.dump(df, r"..\assets\RAWG\RAWG_clean.pkl")

['RAWG_clean.pkl']